---
title: "Some tests with gediDB"
execute: 
  enabled: true
---

In [1]:
import geopandas as gpd
import gedidb as gdb
import plotly.express as px
import plotly.io as pio
from shapely.geometry import Polygon

pio.renderers.default = "iframe"

# Instantiate the GEDIProvider
provider = gdb.GEDIProvider(
    storage_type='s3',
    s3_bucket="dog.gedidb.gedi-l2-l4-v002",
    url="https://s3.gfz-potsdam.de"
)

# Load region of interest (ROI)
region_of_interest = gpd.read_file('MougueirasFire.geojson')

# Define variables to query and quality filters
vars_selected = ["rh_95", 'rh_50']

# Query data
gedi_data_pre = provider.get_data(
    variables=vars_selected,
    query_type="bounding_box",
    geometry=region_of_interest,
    start_time="2018-09-26",
    end_time="2020-09-27",
    return_type='dataframe'
)

gedi_data_post = provider.get_data(
    variables=vars_selected,
    query_type="bounding_box",
    geometry=region_of_interest,
    start_time="2020-09-27",
    end_time="2021-09-27",
    return_type='dataframe'
)

## Change in RH_95 for Oleiros fire in Portugal

In [2]:
#| echo: false
import warnings
warnings.filterwarnings("ignore")

gdf_pre = gpd.GeoDataFrame(gedi_data_pre, geometry=gpd.points_from_xy(x= gedi_data_pre['longitude'], y = gedi_data_pre['latitude']))
gdf_post = gpd.GeoDataFrame(gedi_data_post, geometry=gpd.points_from_xy(x= gedi_data_post['longitude'], y = gedi_data_post['latitude']))

gdf_pre.crs = 4326
gdf_post.crs = 4326

# gedi_ba = gpd.sjoin(gdf_pre, region_of_interest, how='inner').drop(columns='index_right')

# Create a grid over the extent of your points
region_of_interest.to_crs(epsg=32629, inplace=True)  # Project to UTM

bounds = region_of_interest.total_bounds  # Get the bounds of your GeoDataFrame

grid_cells = []

resolution = 250

for x0 in range(int(bounds[0]), int(bounds[2]), resolution):  # Adjust grid size (e.g., 1000 meters)
    for y0 in range(int(bounds[1]), int(bounds[3]), resolution):
        grid_cells.append(Polygon([(x0, y0), (x0 + resolution, y0), (x0 + resolution, y0 + resolution), (x0, y0 + resolution)]))

grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=region_of_interest.crs)

# Spatial join to aggregate points in each grid cell
grid = gpd.sjoin(grid.to_crs(4326), gdf_pre, how='inner').drop(columns='index_right')

aggregated = grid[['geometry', 'rh_95', 'rh_50']].dissolve(by = grid.index, aggfunc='mean').to_crs(epsg=4326)

delta = gpd.sjoin(aggregated, gdf_post).drop(['index_right', 'time', 'shot_number', 'latitude', 'longitude'], axis = 1).reset_index().dissolve('index')

delta['delta_rh_95'] = delta['rh_95_right'] - delta['rh_95_left']
delta['delta_rh_50'] = delta['rh_50_right'] - delta['rh_50_left']

delta =  gpd.sjoin(delta, region_of_interest.to_crs(4326), how='inner').drop(columns='index_right')

fig = px.choropleth_map(delta, geojson = delta.geometry, 
                  locations=delta.index, color='delta_rh_95',
                  color_continuous_scale="RdBu",
                  range_color=(-5, 5), 
                  center = {'lon' : delta.centroid.x.mean(), 'lat' : delta.centroid.y.mean()},
                  zoom = 10)


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()


## Change in RH_50 for Oleiros fire in Portugal

In [3]:
#| echo: false

fig = px.choropleth_map(delta, geojson = delta.geometry, 
                  locations=delta.index, color='delta_rh_50',
                  color_continuous_scale="RdBu",
                  range_color=(-5, 5), 
                  center = {'lon' : delta.centroid.x.mean(), 'lat' : delta.centroid.y.mean()},
                  zoom = 10)


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()